In [1]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()
import base64
import json
import numpy as np
import pandas as pd
from snowflake.connector.pandas_tools import write_pandas
import utils.sf_conn as sf_conn

/var/folders/0n/bwyv05nn3hl95hhlm7qy85fh0000gn/T/ipykernel_42873/255710073.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [3]:
# Encode Client ID and Client Secret
client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

auth_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f"Basic {client_creds_b64}"
}

payload = {
    "grant_type": "client_credentials"
}

auth_response = requests.post(auth_url, headers=headers, data=payload)
auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']

# Step 2: Make API Request with the Access Token
api_headers = {
    "Authorization": "Bearer " + access_token
}

# base_url = 'https://api.spotify.com/v1/'
# track_id = '6y0igZArWVi6Iz0rj35c1Y'
# r = requests.get(base_url + 'audio-features/' + track_id, headers=api_headers)
# r = r.json()
# r


In [13]:
auth_response_data

{'access_token': 'BQDVn0Y0C5BBlKYX0oD0Qo2zwAdEMc_zy0h-QksC9hV64e76RokpBG2heEJ2hOBR_lfGeV_BJia7Wp-reKPHQ2Vej3l3nwWLB5kFxlWnctmXxlzU8s0',
 'token_type': 'Bearer',
 'expires_in': 3600}

In [ ]:
# def get_playlists_name(access_token, playlist_id):
#     url = f"https://api.spotify.com/v1/playlists/{playlist_id}"
#     headers = {'Authorization': f'Bearer {access_token}'}
#     response = requests.get(url, headers=headers)
#     playlist_name = response.json().get("name")


In [ ]:
# url = f"https://api.spotify.com/v1/tracks/6dLLEE96JRnX1ngkL77yiA"
# headers = {'Authorization': f'Bearer {access_token}'}
# response = requests.get(url, headers=headers)
# track_data = response.json()
# artist_popularity = []
# # if "artists" in track_data:
# for artist in track_data["artists"]:
#     print(artist)
#     popularity = artist.get("type")
#     name = artist.get("name", "Unknown Artist")
#     artist_popularity.append((name, popularity))

# # Output the result
# for name, popularity in artist_popularity:
#     print(f"Artist: {name}, Popularity: {popularity}")

In [ ]:
# def get_artist_genres(access_token, artist_id):
#     url = f"https://api.spotify.com/v1/artists/{artist_id}"
#     headers = {'Authorization': f'Bearer {access_token}'}
#     response = requests.get(url, headers=headers)

#     if response.status_code == 200:
#         artist_data = response.json()
#         print(artist_data)
#         return ', '.join(artist_data.get('genres', []))
#     else:
#         print(f"Error fetching genres for artist {artist_id}")
#         return ''

In [ ]:
def get_playlist_name_tracks(access_token, playlist_id):
    all_playlists_data = []
    for ids in playlist_id:
        # Get the playlists name
        url = f"https://api.spotify.com/v1/playlists/{ids}"
        headers = {'Authorization': f'Bearer {access_token}'}
        name_response = requests.get(url, headers=headers)
        playlist_name = name_response.json().get("name")
        # get the track_ids

        url = f'https://api.spotify.com/v1/playlists/{ids}/tracks'
        headers = {'Authorization': f'Bearer {access_token}'}
        track_id_response = requests.get(url, headers=headers)
        
        if track_id_response.status_code == 200:
            track_response = track_id_response.json()
            tracks_data = track_response['tracks']
            for item in tracks_data['items']:
                if item['track'] is not None:
                    track_id = item['track']['id']
                    track_name = item['track']['name']
                    artist_names = ', '.join(artist['name'] for artist in item['track']['artists'])
                    artist_id = ', '.join(artist['id'] for artist in item['track']['artists'])
                 #   genres_list = ', '.join(artist['genres'] for artist in item['track'][0]['track']['artists'][0])
                    #genres = '; '.join(genres_list)

                    all_playlists_data.append({
                        'Playlist_Id': ids,
                        "Playlist_Name": playlist_name,
                        'Track_Name': track_name,
                        "Artist_Names": artist_names,
                        "artist_id":artist_id,
                        "Track_Id": track_id,
                      #  "Genres": genres_list
                    })
        else:
            raise Exception(f"Error in API request: {track_id_response.status_code}")
    playlist_df = pd.DataFrame(all_playlists_data)
    return playlist_df

In [10]:
def get_track_information(access_token, track_id):
    track_data_list = []
    # for id in track_id:
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)
    print(response.headers['access-control-allow-headers'])
    # Check if the response status code indicates success
    if response.status_code == 200:
        try:
            track_data = response.json()
            #print(track_data)
            track_name = track_data.get('name')
            track_popularity = track_data.get('popularity')
            track_uri = track_data.get('uri')

            track_data_list.append({
                
            "Track_Id": track_id,
            "Track_Name": track_name,
            "Track_Popularity": track_popularity,
            "Track_Uri": track_uri
        })

        except:
            print("Error: Could not decode JSON. Response content:", response.text)
            raise
    else:
        # The API request was not successful
        print(f"Error: Received response with status code {response.status_code}. Response content:", response.text)
        return None
    return pd.DataFrame(track_data_list)


In [11]:
track_info = get_track_information(access_token= access_token, track_id = '6dLLEE96JRnX1ngkL77yiA')

track_info

Accept, App-Platform, Authorization, Content-Type, Origin, Retry-After, Spotify-App-Version, X-Cloud-Trace-Context, client-token, content-access-token


,Track_Id,Track_Name,Track_Popularity,Track_Uri
0,6dLLEE96JRnX1ngkL77yiA,"Boom (feat. MOTi, Ty Dolla $ign, Wizkid & Kran...",31,spotify:track:6dLLEE96JRnX1ngkL77yiA


In [ ]:
response.headers

In [ ]:
track_info.info()

In [ ]:
# def get_audio_features(access_token, track_ids):
def get_track_audio_features(access_token, track_ids):
    track_audio_features = []
    for id in track_ids:
        url = f"https://api.spotify.com/v1/audio-features/{id}"
        headers = {'Authorization': f'Bearer {access_token}'}
        response = requests.get(url, headers=headers)
        track = response.json()
        if response.status_code == 200:
            try:
                # for audio in repsone_json['track']:
                audio_data = {
                    'Track_id': id,
                    'danceability': track['danceability'],
                    'duration_ms': track['duration_ms'],
                    'energy': track['energy'],
                    'instrumentalness': track['instrumentalness'],
                    'key': track['key'],
                    'liveness': track['liveness'],
                    'loudness': track['loudness'],
                    'mode': track['mode'],
                    'speechiness': track['speechiness'],
                    'time_signature':track['time_signature'],
                    'track_uri': track['uri']

                }
                track_audio_features.append(audio_data)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON for workbook ID {id}: {e}")
        else:
            raise Exception(f"Error in API request: {response.status_code}")
    track_audio_features_df = pd.DataFrame(track_audio_features)
    return track_audio_features_df

In [ ]:
playlist_data = get_playlist_name_tracks(access_token=access_token, playlist_id=['37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c'])
                                                                            # "37i9dQZF1EIh5VHows1LoY?si=3f242c7e9e7a43f8",
                                                                            # '37i9dQZF1EVKuMoAJjoTIw?si=2ed6df3d293b4519',
                                                                            # '37i9dQZF1EQpoj8u9Hn81e?si=92566118c4954e73',
                                                                            # '37i9dQZF1EQqlvxWrOgFZm?si=2b9ab93cc7714708',
                                                                            # '37i9dQZF1EIhBF9gUW4UsT?si=089940507c6b46f3',
                                                                            # '37i9dQZF1EIherXksVvnrN?si=d8e28d16e28b40b0',
                                                                            # '37i9dQZF1EVHGWrwldPRtj?si=98356d3db1994d67',
                                                                            # "37i9dQZF1DWSf2RDTDayIx?si=1d3f916447094372"
                                                                            # "37i9dQZF1DX0AMssoUKCz7?si=80092935821a4201",
                                                                            # '37i9dQZF1DX0hvSv9Rf41p?si=72b7f2562f0a4020',
                                                                            # "6FSFUWzuF2KigQDKV42Uru?si=cfc88421610b46c3",
                                                                            # '37i9dQZF1E35DbZEmy06Qp?si=b6e38bde3d8d4116'])

In [ ]:
playlist_data.head()

In [ ]:
audio_features_df = get_track_audio_features(access_token = access_token, track_ids = playlist_data['Track_Id'])


In [ ]:
audio_features_df.head()